In [1]:
import pandas as pd
import numpy as np
import csv

### Data gathered from:

- California WARN data: https://edd.ca.gov/Jobs_and_Training/Layoff_Services_WARN.htm
- population data from: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html#par_textimage_242301767

### This notebook will explore layoffs in counties with high rent burdens

In the [finding_ambiguous_cities](https://github.com/biglocalnews/WARN_Cali_analysis/blob/master/hardest_hit/finding_ambiguous_cities.ipynb) notebook, we've already standardized the county information. We've also identified a few records with the incorrect city name, so we need to clean those records. 

In [2]:
cali_layoffs = pd.read_csv('california_warn_raw_2.csv')
cali_population = pd.read_csv('county_population.csv', delimiter='\t', thousands=',')

In [3]:
cali_layoffs.head()

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig
0,0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,Los Angeles County
1,1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,Los Angeles County
2,2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,Los Angeles County
3,3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,Los Angeles County
4,4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,Marin County


In [4]:
cali_layoffs.tail()

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig
9671,9671,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",San Francisco,No Name,86.0,Layoff Permanent,No County Name
9672,9672,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Santa Monica,No Name,49.0,Closure Permanent,No County Name
9673,9673,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Venice,No Name,11.0,Closure Permanent,No County Name
9674,9674,06/29/2015,08/28/2015,06/30/2015,"Safeway, Inc.",Pleasanton,No Name,18.0,Layoff Unknown at this time,No County Name
9675,9675,06/30/2015,07/22/2015,06/30/2015,Medtronic Ablation Frontiers LLC,Carlsbad,No Name,50.0,Closure Permanent,No County Name


### California Layoffs

For the purposes of this analysis, we are selecting 2020 records by isolating the year on the `Notice date` column.
By choosing `Notice date` we make sure that these layoffs were issued after the start of shelter in place for 2020, as some layoff notices can be issues much earlier the layoff date.

In [5]:
cali_layoffs['Year'] = cali_layoffs['Notice Date'].str[-4:]
cali_layoffs.head()

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig,Year
0,0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,Los Angeles County,2020
1,1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,Los Angeles County,2020
2,2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,Los Angeles County,2020
3,3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,Los Angeles County,2020
4,4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,Marin County,2020


### Cleaning Layoff Types

Type Unknown and unknown at this time will be combined.

In [6]:
cali_layoffs['Layoff/Closure'].unique()

array(['Layoff Permanent', 'Layoff Type Unknown ', 'Layoff Temporary',
       'Closure Temporary', 'Closure Permanent', 'Closure Type Unknown ',
       'Closure Unknown at thistime', 'Layoff Unknown at thistime',
       'Layoff Unknown at this time', 'Closure Unknown at this time'],
      dtype=object)

In [7]:
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure'].str.replace('at thistime', 'at this time')
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.strip()
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.lower()
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.replace('unknown at this time', 'type unknown')
cali_layoffs['Layoff/Closure clean'].unique()

array(['layoff permanent', 'layoff type unknown', 'layoff temporary',
       'closure temporary', 'closure permanent', 'closure type unknown'],
      dtype=object)

In [8]:
years = ['2020', '2019']

### Extracting Only Necessary Records

Now that the data has been standardized, we will be selecting the records that have a notice date from the year 2020 or 2019. This will allow for comparisons between a pandemic and non-pandemic year in terms of layoff notices. 

In [9]:
cali_2020_2019 = cali_layoffs[cali_layoffs["Year"].isin(years)]
cali_2020_2019.head()

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean
0,0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
1,1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
2,2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
3,3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
4,4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,Marin County,2020,layoff permanent


In [18]:
# Checking for incorrect County in 'County' Column
# pd.options.display.max_rows = 999
# cali_2020_2019[cali_2020_2019['City'] == 'Sacramento']

### More Data Cleaning

A few records have the incorrect county, based on the information in the city column. These cities include: Sacramento, Hayward, Indian Wells, Cerritos, Vista, Roseville, City of Industry, Del Mar, Walnut Creek, and Los Gatos. For information on how these cities were selected, see the [finding_ambiguous_cities](https://github.com/biglocalnews/WARN_Cali_analysis/blob/master/hardest_hit/finding_ambiguous_cities.ipynb) notebook.

In [11]:
dict_cities = {
    'Sacramento':'Sacramento County',
    'San Francisco':'San Francisco County',
    'Los Angeles': 'Los Angeles County',
    'Hayward':'Alameda County',
    'Indian Wells':'Riverside County',
    'Cerritos':'Los Angeles County',
    'Vista':'San Diego County',
    'Roseville':'Placer County',
    'Berkeley': 'Alameda County',
    'City of Industry':'Los Angeles County',
    'Del Mar':'San Diego County',
    'Walnut Creek':'Contra Costa County',
    'Los Gatos':'Santa Clara County'
}

### Clean County Names
Below is our code to correct County Names based on City Names. 
We are taking the dictionary above, looping through the city and county names, which are the keys and values, replacing the incorrect County Name with the correct one for each city's record.

In [12]:
for city, county in dict_cities.items():
    cali_2020_2019.loc[cali_2020_2019['City'] == city, ['County']] = county

/Users/dilcia_mercedes/.local/share/virtualenvs/cali_WARN-MY_IDcvv/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [13]:
# Test to make sure Counties are named correctly. All records in 'County' column should now say sacramento - the 'County Orig' column should have the original County stated
sacramento_clean = cali_2020_2019[cali_2020_2019['City'] == 'Sacramento']
sacramento_clean.loc[1962:1963]

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean
1962,1962,03/17/2020,03/19/2020,06/05/2020,"Chico Community Publishing, Inc.",Sacramento,Sacramento County,45.0,Layoff Permanent,Del Norte County,2020,layoff permanent


In [14]:
cali_2020_2019.drop(columns='Unnamed: 0', inplace=True)
cali_2020_2019.head()

/Users/dilcia_mercedes/.local/share/virtualenvs/cali_WARN-MY_IDcvv/lib/python3.8/site-packages/pandas/core/frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean
0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,Marin County,2020,layoff permanent


### Exporting Standardized Data

Now that we have 2020 and 2019 data in an isolated dataframe, we can export it and clean the company names in Open Refine.
The data with clean company names will be re-read into the notebook on the line after the export. 

In [15]:
# Export File to clean company names in open refine.
cali_2020_2019.to_csv('tobecleaned_california_warn.csv')

In [16]:
### Re-import clean company names file.
# open_refine_clean = pd.read_csv('cleaned_california_warn.csv')
# open_refine_clean.head()

### Adding Population Data

The data has been grabbed from the census link above. These figures will be used to calculate per capita layoff numbers per county.

In [17]:
# cali_population.rename(columns={'California':'Counties','39,512,223':'Population'}, inplace=True)
# cali_population['County Names'] = cali_population['Counties'].str[1:]
# cali_population['County Names'] = cali_population['County Names'].str.split(',').str[0]
# cali_population.head(4)